In [1]:
import torch
from torchvision import models

In [2]:
x = torch.randn(1, 3, 224, 224)

In [4]:
model = models.shufflenet_v2_x0_5(weights=models.ShuffleNet_V2_X0_5_Weights.DEFAULT)

In [7]:
onnx_path = "shufflenet_v2_x0_5.onnx"
torch.onnx.export(
    model=model,
    args=x,
    opset_version=17,
    f=onnx_path,
    input_names=["images"],
    output_names=["classes"]
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [6]:
import onnx
from onnxsim import simplify

In [ ]:
model_ = onnx.load(onnx_path)

In [10]:
model__, ok = simplify(model_)
print(ok)

True


In [11]:
onnx.save(model__, onnx_path)